In [15]:
# data import and cleaning
import numpy as np

with open(f"../data/data.txt", 'r', encoding='utf-8-sig') as f:
    file_list = f.readlines()

    pointer = 0
    for i, line in enumerate(file_list):
        file_list[i] = [float(item) for item in line.replace(' ', '').split(',')]

    print(file_list)
    # first line is number of warehouses
    num_warehouses = int(file_list[pointer][0])
    pointer += 1

    # second line is number of goods
    num_goods = int(file_list[pointer][0])
    pointer += 1

    # third line is number of retailers
    num_retailers = int(file_list[pointer][0])
    pointer += 1

    # fourth line is the capacity of each warehouse
    warehouse_capacities = [int(value) for value in file_list[pointer]]
    assert len(warehouse_capacities) == num_warehouses
    pointer += 1

    # next block is the demand of each retailer for each good and the shortage unit cost
    retailer_good_demand = np.ndarray(shape=(num_retailers, num_goods))
    retailer_good_shortage_cost = np.ndarray(shape=(num_retailers, num_goods))
    for line in file_list[pointer:pointer + num_retailers * num_goods]:
        retailer = int(line[0]) - 1
        good = int(line[1]) - 1
        demand = int(line[2])
        shortage_cost = line[3]
        retailer_good_demand[retailer, good] = demand
        retailer_good_shortage_cost[retailer, good] = shortage_cost

    pointer += num_retailers * num_goods

    # next line is the sizes of each of the goods
    good_sizes = [int(value) for value in file_list[pointer]]
    assert len(good_sizes) == num_goods
    pointer += 1

    # next line is the holding cost of each good
    good_holding_costs = file_list[pointer]
    assert len(good_holding_costs) == num_goods
    pointer += 1

    # next block is the initial stock of each good at each warehouse
    # we initialise as zeroes in case the file doesn't include zero stock goods
    warehouse_good_initial_stock = np.zeros(shape=(num_warehouses, num_goods))
    for line in file_list[pointer:pointer + num_warehouses * num_goods]:
        warehouse = int(line[0]) - 1
        good = int(line[1]) - 1
        initial_stock = int(line[2])
        warehouse_good_initial_stock[warehouse, good] = initial_stock

    pointer += num_warehouses * num_goods

    # the last block is the unit transportation cost for each good between each retailer and each warehouse
    transportation_costs = np.ndarray(shape=(num_retailers, num_warehouses, num_goods))
    for line in file_list[pointer:pointer + num_warehouses * num_retailers * num_goods]:
        warehouse = int(line[0]) - 1
        retailer = int(line[1]) - 1
        good = int(line[2]) - 1
        transport_cost = line[3]
        transportation_costs[warehouse, retailer, good] = transport_cost

print(f"""Data loaded successfully!
    - num_warehouses: {num_warehouses}
    - num_goods: {num_goods}
    - num_retailers {num_retailers}
    - warehouse_capacities: {warehouse_capacities}
    - retailer_good_demand:\n {retailer_good_demand}
    - retailer_good_shortage_cost:\n {retailer_good_shortage_cost}
    - good_sizes: {good_sizes}
    - good_holding_costs: {good_holding_costs}
    - warehouse_good_initial_stock:\n {warehouse_good_initial_stock}
    - transportation_costs:\n {transportation_costs}
""")

[[5.0], [3.0], [5.0], [800.0, 1000.0, 700.0, 1000.0, 600.0], [1.0, 1.0, 200.0, 2.5], [1.0, 2.0, 150.0, 3.0], [1.0, 3.0, 350.0, 2.0], [2.0, 1.0, 250.0, 3.0], [2.0, 2.0, 200.0, 2.0], [2.0, 3.0, 400.0, 3.0], [3.0, 1.0, 150.0, 2.0], [3.0, 2.0, 300.0, 1.5], [3.0, 3.0, 200.0, 1.5], [4.0, 1.0, 300.0, 1.5], [4.0, 2.0, 150.0, 3.0], [4.0, 3.0, 150.0, 3.0], [5.0, 1.0, 200.0, 2.5], [5.0, 2.0, 100.0, 2.5], [5.0, 3.0, 200.0, 2.5], [1.0, 0.8, 0.6], [2.0, 5.0, 3.0], [1.0, 1.0, 300.0], [1.0, 2.0, 200.0], [1.0, 3.0, 300.0], [2.0, 1.0, 400.0], [2.0, 2.0, 300.0], [2.0, 3.0, 300.0], [3.0, 1.0, 200.0], [3.0, 2.0, 100.0], [3.0, 3.0, 200.0], [4.0, 1.0, 50.0], [4.0, 2.0, 150.0], [4.0, 3.0, 300.0], [5.0, 1.0, 150.0], [5.0, 2.0, 150.0], [5.0, 3.0, 200.0], [1.0, 1.0, 1.0, 0.6], [1.0, 1.0, 2.0, 0.8], [1.0, 1.0, 3.0, 1.0], [1.0, 2.0, 1.0, 0.7], [1.0, 2.0, 2.0, 0.9], [1.0, 2.0, 3.0, 1.0], [1.0, 3.0, 1.0, 1.3], [1.0, 3.0, 2.0, 1.5], [1.0, 3.0, 3.0, 0.8], [1.0, 4.0, 1.0, 1.4], [1.0, 4.0, 2.0, 1.3], [1.0, 4.0, 3.0, 2.0

In [ ]:
import gurobipy as gp

m = gp.Model()

# modelling stuff goes here